# Recommender for Amazon Books with Matrix Factorisation



## Load data into SQLite database


In [6]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine # database connection
import datetime as dt

disk_engine = create_engine('sqlite:///ratings.db') # Initializes database with filename 311_8M.db in current directory


def load_data(chunksize = 20000):
    start = dt.datetime.now()
    j = 0
    index_start = 1
    data_columns = ['UserId', 'ItemId', 'Rating', 'Time']

    for df in pd.read_csv('dist/ratings_Amazon_Instant_Video.csv', names=data_columns, chunksize=chunksize, iterator=True, encoding='utf-8'):
        df.index += index_start
        j+=1
        print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

        df.to_sql('data', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1

#load_data()

0 seconds: completed 20000 rows
0 seconds: completed 40000 rows
0 seconds: completed 60000 rows
0 seconds: completed 80000 rows
0 seconds: completed 100000 rows
1 seconds: completed 120000 rows
1 seconds: completed 140000 rows
1 seconds: completed 160000 rows
1 seconds: completed 180000 rows
2 seconds: completed 200000 rows
2 seconds: completed 220000 rows
2 seconds: completed 240000 rows
2 seconds: completed 260000 rows
3 seconds: completed 280000 rows
3 seconds: completed 300000 rows
3 seconds: completed 320000 rows
3 seconds: completed 340000 rows
4 seconds: completed 360000 rows
4 seconds: completed 380000 rows
4 seconds: completed 400000 rows
4 seconds: completed 420000 rows
5 seconds: completed 440000 rows
5 seconds: completed 460000 rows
5 seconds: completed 480000 rows
5 seconds: completed 500000 rows
6 seconds: completed 520000 rows
6 seconds: completed 540000 rows
6 seconds: completed 560000 rows
6 seconds: completed 580000 rows
6 seconds: completed 600000 rows


## Query data from database

In [15]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

disk_engine = create_engine('sqlite:///ratings.db') # Initializes database with filename 311_8M.db in current directory

df = pd.read_sql_query('SELECT `UserId`, `ItemId`, `Rating` FROM `data` WHERE `UserId` IN (SELECT `UserId` '
                       'FROM `data`'
                       'GROUP BY `UserId`'
                       'HAVING COUNT(`Rating`) > 20)', disk_engine)

R_df = df.pivot(index = 'UserId', columns ='ItemId', values = 'Rating').fillna(0)
R_df.head()

ItemId,B000GIOPK2,B000H00VBQ,B000H0YRNY,B000H2DMME,B000H2DTWM,B000H4YNM0,B000HAB4NK,B000HKWE3O,B000HMPU0Q,B000HZEHL6,...,B00LEAOLBY,B00LERQAB6,B00LFE2SX2,B00LG0DKBO,B00LGIO8X0,B00LIRFK5E,B00LM493J2,B00LPWPMCS,B00LSWLQQQ,B00LTMJ29S
UserId,,,,,,,,,,,,,,,,,,,,,
A109ME7C09HM2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10L7F2V8368DO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11A75FIE3396D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11KYS8T5NLKZ1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11Q7PYQ87R5Y2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Play with matrices

In [19]:
from scipy.sparse.linalg import svds

R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

